In [3]:
!pip install faker

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 508.0 kB/s eta 0:00:03
   ---------------- ----------------------- 0.8/1.9 MB 633.2 kB/s eta 0:00:02
   ---------------- ----------------------- 0.8/1.9 MB 633.2 kB/s eta 0:00:02
   --------------------- ------------------ 1.0/1.9 MB 653.7 kB/s eta 0:00:02
   --------------------- ------------------ 1.0/1.9 MB 653.7 kB/s eta 0:00:02
   --------------------- ------------------ 1.0/1.9 MB 653.7 kB/s eta 0:00:02
   --------------------------- ------------ 1.3/1

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\faker.exe' -> 'C:\\Python312\\Scripts\\faker.exe.deleteme'


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import pandas as pd 
import random 
from faker import Faker 
from datetime import datetime, timedelta

# generate tbl_branch
# Inisialisasi Faker 
fake = Faker()
# Jumlah data 
num_records = 50 
# dummy data 
branch_data ={
    "id":[i + 1 for i in range(num_records)],
    "name":[fake.company() for _ in range(num_records)],
    "location":[fake.city() for _ in range(num_records)],
    "address":[fake.address() for _ in range(num_records)],
    "email":[fake.email() for _ in range(num_records)],
    "phone":[fake.phone_number() for _ in range(num_records)],
    "created_time":[fake.date_time_this_decade() for _ in range(num_records)],
    "modified_time":[fake.date_time_this_decade() for _ in range(num_records)]
}
df_branch = pd.DataFrame(branch_data)


# generate tbl_employee
employee_data = []
# working_hours = [(7,15), (10,18), (14,22)]
global_employee_id = 1

for branch_id in branch_data["id"]:
    employee_list = []
    total_employees = random.randint(5,10)
    active_count = 0 

    for  _ in range(total_employees):
        created_at = fake.date_time_this_decade()
        modified_at = created_at + timedelta(days=random.randint(0,30))

        # Pastikan minimal 3 karyawan aktif
        if active_count < 3:
            active_status = True
            active_count +=1
        else:
            active_status = random.choice([True, False])
            if active_status:
                active_count += 1
        
        employee_list.append({
                "id": global_employee_id,
                "branch_id": branch_id,
                "name": fake.name(),
                "salary": round(random.uniform(3000000, 8000000),2),
                "active": active_status,
                "address": fake.address(),
                "phone": fake.phone_number(),
                "email": fake.email(),
                "created_at": fake.date_time_this_decade(),
            })
        
        global_employee_id += 1

    employee_data.extend(employee_list)

df_employee = pd.DataFrame(employee_data)



# generate tbl schedulle
# function for get day from date 
def get_day_of_week(date):
    return date.strftime('%A') # get name from date

# define shift 
#shifts = [(7,12), (12,17), (17,22)]

schedule_data = []
global_schedule_id = 1

start_date = datetime(datetime.now().year, 1,1)
end_date = datetime(datetime.now().year,12,31)
delta = end_date - start_date


for branch_id in branch_data["id"]:
    active_employees = [emp for emp in employee_data if emp["branch_id"] == branch_id and emp["active"]]
    if len(active_employees)< 3:
        continue # skip branch if active_employees kurang dari 3

    for day in range(delta.days + 1):
        current_date = start_date + timedelta(days= day)

        # get day from date 
        day_of_week = get_day_of_week(current_date)

        # define jumlah jaga based on weekday or weekend 
        if day_of_week in ["Saturday", "Sunday"]: 
            num_employees = min(2, len(active_employees))
            shifts = [(10,15), (15,20)]
            shifts = [(7,12), (12,17), (17,22)]
        else:
            num_employees = len(active_employees)
            shifts = [(7,12), (12,17), (17,22)]

        # distribusi shift karyawan 
        shift_capacities = [num_employees // len(shifts)] * len(shifts)
        for i in range(num_employees % len(shifts)):
            shift_capacities[i] +=1 # tambahkan sisa karyawan ke shift pertama    

        employees_left = active_employees[:]
        shift_assignments = []

        for i, (shift_start, shift_end)  in enumerate(shifts):
            if shift_capacities[i] > 0:
                assigned_employees = random.sample(employees_left, shift_capacities[i]) # get random employee for shift
                shift_assignments.append((assigned_employees, f"{shift_start}:00 - {shift_end}:00"))
                employees_left = [emp for emp in employees_left if emp not in assigned_employees] # delete assign employee from list employee

        for assigned_employees, shift_time in shift_assignments:
            for emp in assigned_employees:
                schedule_data.append({
                    "id": global_schedule_id,
                    "branch_id": branch_id,
                    "employee_id": emp["id"],
                    "day": day_of_week,
                    "date": current_date.date(),
                    "jam_shift": shift_time,
                    "created_time": fake.date_time_this_decade(),
                })
                global_schedule_id +=1

df_schedule = pd.DataFrame(schedule_data)


    
with pd.ExcelWriter("dummy_data3.xlsx") as writer:
    df_branch.to_excel(writer, sheet_name="Branch", index=False)
    df_employee.to_excel(writer, sheet_name="Employee", index=False)
    df_schedule.to_excel(writer, sheet_name="Schedule", index=False)

print("Dummy data berhasil dibuat dan disimpan sebagai dummy_data.xlsx")
    





Dummy data berhasil dibuat dan disimpan sebagai dummy_data.xlsx


In [35]:
df_schedule

,id,branch_id,employee_id,day,date,jam_shift,created_time
0,1,1,1,Wednesday,2025-01-01,7:00 - 12:00,2023-04-17 09:52:23
1,2,1,3,Wednesday,2025-01-01,12:00 - 17:00,2022-08-18 08:48:39
2,3,1,2,Wednesday,2025-01-01,17:00 - 22:00,2024-03-15 11:07:41
3,4,1,2,Thursday,2025-01-02,7:00 - 12:00,2020-10-20 17:05:09
4,5,1,3,Thursday,2025-01-02,12:00 - 17:00,2025-01-03 01:49:43
...,...,...,...,...,...,...,...
75123,75124,50,347,Wednesday,2025-12-31,7:00 - 12:00,2020-10-30 16:40:58
75124,75125,50,348,Wednesday,2025-12-31,12:00 - 17:00,2020-06-14 16:31:05
75125,75126,50,341,Wednesday,2025-12-31,12:00 - 17:00,2024-05-28 04:19:16
75126,75127,50,343,Wednesday,2025-12-31,17:00 - 22:00,2022-12-18 13:04:38


In [2]:
import pandas as pd 
from datetime import datetime 

# product data 

data_product = [
    (1, "iPhone 15 Pro", "Electronics", "Smartphones", 15000000, 5, 10),
    (2, "Samsung Galaxy S23", "Electronics", "Smartphones", 20000000, 5, 5),
    (3, "MacBook Pro M3", "Electronics", "Laptops", 22000000, 5, 20),
    (4, "Asus Zenbook Flip", "Electronics", "Laptops", 11000000, 5, 20),
    (5, "Asus ROG Zephyrus G14", "Electronics", "Laptops", 15000000, 5, 30),
    (6, "JBL Flip 6", "Electronics", "Speakers", 3000000, 10, 100),
    (7, "Logitech MX Master 3", "Electronics", "Accessories", 1700000, 20, 200),
    (8, "Air Jordan 1 Retro High Dior", "Apparel", "Footwear", 125000000, 2, 5),
    (9, "Adidas Ultraboost", "Apparel", "Footwear", 3000000, 10, 20),
    (10, "versace chain contour", "Apparel", "Clothing", 2700000, 10, 20),
    (11, "Rolex Submariner", "Fashion", "Watches", 150000000, 2, 5),
    (12, "Fossil Leather Wallet", "Fashion", "Accessories", 1000000, 5, 10),
    (13, "Samsung 65\" Smart TV", "Home Appliances", "Televisions", 10000000, 5, 40),
    (14, "Dyson V15 Vacuum", "Home Appliances", "Cleaning Devices", 2000000, 10, 200),
    (15, "Philips Air Fryer", "Home Appliances", "Kitchen Appliances", 700000, 10, 200),
]

created_time = datetime.today().strftime('%Y-%m-%d')

df_product = pd.DataFrame(data_product, columns=["id", "product_name", "category", "sub_category", "price", "profit", "stock"])
df_product["created_time"] = created_time

df_product["modified_time"] = None



In [3]:
import pandas as pd 
import random 
from datetime import datetime, timedelta

special_events = {
    "2025-01-01": "New Year",
    "2025-02-10": "Chinese New Year",
    "2025-02-14": "Valentine's Day",
    "2025-03-08": "International Women's Day",
    "2025-03-11": "Nyepi Day",
    "2025-03-17": "St. Patrick's Day",
    "2025-03-29": "Good Friday",
    "2025-04-01": "April Fool's Day",
    "2025-04-10": "Eid al-Fitr",
    "2025-04-11": "Eid al-Fitr Holiday",
    "2025-04-21": "Kartini Day",
    "2025-05-01": "Labor Day",
    "2025-05-09": "Ascension Day of Jesus Christ",
    "2025-06-01": "Pancasila Day",
    "2025-06-17": "Eid al-Adha",
    "2025-07-04": "US Independence Day",
    "2025-07-07": "Islamic New Year",
    "2025-08-17": "Independence Day",
    "2025-09-16": "Prophet Muhammad’s Birthday",
    "2025-10-31": "Halloween",
    "2025-11-10": "Heroes Day",
    "2025-11-29": "Black Friday",
    "2025-12-24": "Christmas Eve",
    "2025-12-25": "Christmas Day",
    "2025-12-26": "Boxing Day",
    "2025-12-31": "New Year's Eve"
}

current_year = datetime.now().year
# add event setiap akhir bulan 
for month in range(1,13):
    last_day = datetime(current_year, month, 1) + timedelta(days=32)
    last_day = last_day.replace(day=1) - timedelta(days=1)
    special_events[last_day.strftime("%Y-%m-%d")] = f"End of {last_day.strftime('%B')} Sale"

promotion_data = []
promo_id = 1 

for date, event in sorted(special_events.items()):
    promotion_data.append({
        "id": promo_id,
        "event_name": f"{event} ({date})",
        "disc": random.randint(5,10),
        "time": date,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })
    promo_id +=1

df_promotions = pd.DataFrame(promotion_data)


In [4]:
import pandas as pd 
import random 
import faker 
from datetime import datetime, timedelta

fake = faker.Faker()

num_records_cust = 100

data_cust = []
for i in range(1, num_records_cust + 1):
    name = fake.name()
    address = fake.address().replace("\n", ", ")
    phone = fake.phone_number()
    email = fake.email()
    created_at = fake.date_time_between(start_date="-2y", end_date="now").strftime("%Y-%m-%d %H:%M:%S")
    modified_at = (datetime.strptime(created_at, "%Y-%m-%d %H:%M:%S") + timedelta(days=random.randint(1, 365))).strftime("%Y-%m-%d %H:%M:%S")
    data_cust.append([i, name, address, phone, email, created_at, modified_at])

df_cust = pd.DataFrame(data_cust, columns=["id", "name", "address", "phone", "email", "created_at", "modified_at",])


In [5]:
import pandas as pd 
import random 
from datetime import datetime, timedelta

data_payment_method = {
    (1, "Cash Payment"),
    (2, "Bank Transfer"),
    (3, "Credit Card"),
    (4, "Virtual Account")
}
data_order_status = {
    (1, "ongoing"),
    (2, "done")
}
data_payment_status = {
    (1, "processing"),
    (2, "paid"),
    (3, "failed")
}
data_shipping_status = {
    (1, "ongoing"),
    (2, "done"),
    (3, "failed")
}


created_time = datetime.today().strftime('%Y-%m-%d')
df_payment_method = pd.DataFrame(data_payment_method, columns = ["id", "name"])
df_order_status = pd.DataFrame(data_order_status, columns=["id", "name"])
df_payment_status = pd.DataFrame(data_payment_status, columns=["id", "name"])
df_shipping_status = pd.DataFrame(data_shipping_status, columns=["id", "name"])

df_payment_method["created_time"] = created_time
df_order_status["created_time"] = created_time
df_payment_status["created_time"] = created_time
df_shipping_status["created_time"] = created_time

In [6]:
import pandas as pd 
import random 
import faker
from datetime import datetime, timedelta

fake = faker.Faker()
customer_ids = df_cust["id"].tolist()
product_ids = df_product["id"].tolist()
branch_ids = df_branch["id"].tolist()

transactions_per_month = random.randint(500, 1000)

order_status_options = [1, 2]
order_status_weghts = [0.1, 0.9]

payment_status_options = [1, 2, 3]
payment_status_weights = [0.05, 0.9, 0.05]

payment_method_options = [1, 2, 3, 4]

shipping_status_options = [1, 2, 3]
shipping_status_weights = [0.05, 0.9, 0.05]

sales_data = []
sale_id = 1

def get_random_order_time(date):
    if date.weekday() < 5:  # Weekday (Monday-Friday)
        hour = random.randint(7, 22)
    else:  # Weekend (Saturday-Sunday)
        hour = random.randint(10, 20)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    return datetime(date.year, date.month, date.day, hour, minute, second)

for month in range(1, 13):
    start_date = datetime(2025, month, 1)
    end_date = (start_date + timedelta(days=32)).replace(day=1) - timedelta(days=1)

    for _ in range(transactions_per_month):
        product_id = random.choice(product_ids)
        customer_id = random.choice(customer_ids)
        branch_id = random.choice(branch_ids)
        quantity = random.randint(1,10)
        payment_method = random.choice(payment_method_options)
        is_online_transaction = random.choice([True, False])
        delivery_fee = 0
        is_free_delivery_fee = None 

        if is_online_transaction: 
            delivery_fee = random.randint(12000, 50000)
            is_free_delivery_fee = random.choice([True, False])

        random_date = fake.date_between(start_date=start_date, end_date=end_date)
        order_date = get_random_order_time(random_date).strftime("%Y-%m-%d %H:%M:%S")
        
        order_status = random.choices(order_status_options, weights=order_status_weghts, k=1)[0]
        payment_status = None
        shipping_status = None

        if order_status == 2:
            payment_status = random.choices(payment_status_options, weights=payment_status_weights, k=1)[0]
            if payment_status == 2:
                shipping_status = random.choices(shipping_status_options, weights=shipping_status_weights, k=1)[0]
       
        if not is_online_transaction:
            shipping_status = None 
        
        created_at = order_date
        modified_at = (datetime.strptime(created_at, "%Y-%m-%d %H:%M:%S") + timedelta(days=random.randint(1,30))).strftime("%Y-%m-%d %H:%M:%S")

        sales_data.append([sale_id, product_id, customer_id, branch_id, quantity, payment_method, order_date,
                           order_status, payment_status, shipping_status, is_online_transaction, delivery_fee, is_free_delivery_fee, created_at, modified_at])
        sale_id +=1

df_sales = pd.DataFrame(sales_data, columns=["id", "product_id", "customer_id", "branch_id", "quantity", "payment_method", "order_date",
                                              "order_status", "payment_status", "shipping_status","is_online_transaction", "delivery_fee", "is_free_delivery_fee", "created_at", "modified_at"])


In [7]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_sales"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
product_id INT not null,
customer_id int not null, 
branch_id int not null, 
quantity int not null, 
payment_method int, 
order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
order_status int, 
payment_status int, 
shipping_status int, 
is_online_transaction varchar, 
delivery_fee int,
is_free_delivery_fee varchar,
created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
modified_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_sales.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


944

In [29]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_branch"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
name varchar not null, 
location varchar not null, 
address varchar, 
email varchar, 
phone varchar, 
created_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
modified_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_branch.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


50

In [ ]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_product"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
product_name varchar,
category varchar, 
sub_category varchar, 
price integer,
profit int, 
stock int, 
created_time date, 
modified_time date null
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_product.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


15

In [39]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_shipping_status"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
name varchar,
created_time date
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_shipping_status.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


3

In [37]:
df_payment_status

,id,name,created_time
0,1,processing,2025-02-20
1,3,failed,2025-02-20
2,2,paid,2025-02-20


In [42]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_employee"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
branch_id int not null, 
name varchar,
salary integer, 
active varchar, 
address varchar, 
phone varchar, 
email varchar, 
created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_employee.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


393

In [45]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_schedule_employee"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
branch_id int not null, 
employee_id int not null, 
day varchar, 
date date,
jam_shift varchar,
created_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_schedule.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


87

In [49]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_promotions"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
event_name varchar not null, 
disc int not null, 
time date not null, 
created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_promotions.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


36

In [52]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine, text 

# configurasi connection PostgreSQL 
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "tbl_customers"

# creatate connection to PostgreSQL

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME};"
cur.execute(create_schema_query)

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
id INT PRIMARY KEY,
name varchar not null, 
address varchar, 
phone varchar, 
email varchar, 
created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
modified_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()


df_cust.to_sql(TABLE_NAME, engine, schema=SCHEMA_NAME, if_exists='append', index=False)


100

In [1]:
!pip install confluent_kafka

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 1.3 MB/s eta 0:00:03
   ------- -------------------------------- 0.8/4.0 MB 907.1 kB/s eta 0:00:04
   ------- -------------------------------- 0.8/4.0 MB 907.1 kB/s eta 0:00:04
   ------- -------------------------------- 0.8/4.0 MB 907.1 kB/s eta 0:00:04
   ------- -------------------------------- 0.8/4.0 MB 907.1 kB/s eta 0:00:04
   ---------- ----------------------------- 1.0/4.0 MB 592.2 kB/s eta 0:00:05
   ---------- ----------------------------- 1.0/4.0 MB 592.2 kB/s eta 0:00:05
   ------------- -------------------------- 1.3/4.0 MB 578.4 kB/s eta 0:00:05
   ------------- -------------------------- 1.3/4.0 MB 578.4 kB/s eta 0:00:05
   ------------- -------------------------- 1.3/4.0 MB 578.4 kB/s eta 0:00:05
   ------------


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import json 
import time
import uuid 
import random 
from datetime import datetime 
from faker import Faker
from confluent_kafka import Producer
import psycopg2

fake = Faker()
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "tbl_sales_new"


producer_conf = {'bootstrap.servers': KAFKA_BROKER}
producer = Producer(producer_conf)
# Konfigurasi PostgreSQL

POSTGRES_CONFIG = {
    "host": "localhost",
    "database": "sales-project",
    "user": "postgres",
    "password": "postgres"
}

def get_list(column, tabel):
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute(f"select {column} from {tabel} order by {column}")
        products = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
        return products
    except Exception as e:
        print(f"❌ Gagal mengambil produk dari PostgreSQL: {e}")
        return []
    
def get_max_id():
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute("select coalesce(max(id), 0) from tbl_sales")
        max_id = cur.fetchone()[0]
        cur.close()
        conn.close()
        return max_id
    except Exception as e:
        print(f"❌ Gagal mengambil max transaction_id dari PostgreSQL: {e}")
        return 0
    

product_list = get_list('id', 'tbl_product')
customer_list = get_list('id', 'tbl_customers')
branch_list = get_list('id', 'tbl_branch')
payment_method_list = get_list('id', 'tbl_payment_method')

order_status_list = get_list('id', 'tbl_order_status')
order_status_list_weights = [0.1, 0.9]

payment_status_list = get_list('id', 'tbl_payment_status')
payment_status_list_weights = [0.05, 0.9, 0.05]

shipping_status_list = get_list('id', 'tbl_shipping_status')
shipping_status_list_weights = [0.05, 0.9, 0.05]



def generate_sales_data():
    num_rows = random.randint(1,5)
    sales_data = []
    start_id = get_max_id()
    
    for i in range(num_rows):

        order_status =  random.choices(order_status_list, weights=order_status_list_weights, k=1)[0]
        payment_status = None
        shipping_status = None
        is_online_transaction = random.choice([True, False])
        delivery_fee = 0
        is_free_delivery_fee = None 

        if is_online_transaction: 
            delivery_fee = random.randint(12000,50000)
            is_free_delivery_fee = random.choice([True, False])


        if order_status == 2:
            payment_status = random.choices(payment_status_list, weights=payment_status_list_weights, k=1)[0]
            if payment_status ==2:
                shipping_status = random.choices(shipping_status_list, weights=shipping_status_list_weights, k=1)[0]
        
        if not is_online_transaction:
            shipping_status = None
        created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


        sale = {
            "id":start_id + i +1,
            "product_id": random.choice(product_list),
            "customer_id": random.choice(customer_list),
            "branch_id": random.choice(branch_list),
            "quantity": random.randint(1,5),
            "payment_method": random.choice(payment_method_list),
            "order_date": created_at,
            "order_status": order_status,
            "payment_status": payment_status,
            "shipping_status": shipping_status,
            "is_online_transaction": is_online_transaction,
            "delivery_fee": delivery_fee,
            "is_free_delivery_fee": is_free_delivery_fee,
            "created_at":created_at,
            "modified_at": created_at,
        }
        sales_data.append(sale)
    return sales_data

def delivery_report(err, msg):
    """Callback untuk menangani hasil pengiriman Kafka"""
    if err is not None:
        print(f"❌ Gagal mengirim pesan: {err}")
    else:
        print(f"✅ Data dikirim ke Kafka: {msg.topic()} [{msg.partition()}]")

while True:
    sales_data = generate_sales_data()
    for record in sales_data:
        producer.produce(TOPIC_NAME, json.dumps(record).encode("utf-8"), callback=delivery_report)
    producer.flush()

    print(f"✅ {len(sales_data)} data berhasil dikirim ke Kafka!")
    time.sleep(60)  # Tunggu 1 menit sebelum mengirim data baru
        





✅ Data dikirim ke Kafka: tbl_sales [0]
✅ Data dikirim ke Kafka: tbl_sales [0]
✅ Data dikirim ke Kafka: tbl_sales [0]
✅ 3 data berhasil dikirim ke Kafka!


In [ ]:
import json 
import random 
import json 
import time
import uuid 
import random 
from datetime import datetime 
from faker import Faker
from confluent_kafka import Producer
import psycopg2

fake = Faker()
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "tbl_sales_schema_new"


producer_conf = {'bootstrap.servers': KAFKA_BROKER}
producer = Producer(producer_conf)
# Konfigurasi PostgreSQL

POSTGRES_CONFIG = {
    "host": "localhost",
    "database": "sales_project",
    "user": "postgres",
    "password": "postgres"
}

def get_list(column, tabel):
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute(f"select {column} from {tabel} order by {column}")
        products = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
        return products
    except Exception as e:
        print(f"❌ Gagal mengambil produk dari PostgreSQL: {e}")
        return []
    
def get_max_id():
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute("select coalesce(max(id), 0) from tbl_sales")
        max_id = cur.fetchone()[0]
        cur.close()
        conn.close()
        return max_id
    except Exception as e:
        print(f"❌ Gagal mengambil max transaction_id dari PostgreSQL: {e}")
        return 0
    

product_list = get_list('id', 'tbl_product')
customer_list = get_list('id', 'tbl_customers')
branch_list = get_list('id', 'tbl_branch')
payment_method_list = get_list('id', 'tbl_payment_method')

order_status_list = get_list('id', 'tbl_order_status')
order_status_list_weights = [0.1, 0.9]

payment_status_list = get_list('id', 'tbl_payment_status')
payment_status_list_weights = [0.05, 0.9, 0.05]

shipping_status_list = get_list('id', 'tbl_shipping_status')
shipping_status_list_weights = [0.05, 0.9, 0.05]


def generate_sales_data():
    num_rows = random.randint(1,5)
    sales_data = []
    start_id = get_max_id()

    for i in range(num_rows):
        id = start_id + i +1
        product_id = random.choice(product_list)
        customer_id = random.choice(customer_list)
        branch_id = random.choice(branch_list)
        quantity = random.randint(1,5)
        payment_method = random.choice(payment_method_list)
        order_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        order_status = random.choices(order_status_list, weights=order_status_list_weights, k=1)[0]
        payment_status = None
        shipping_status = None 
        is_online_transaction = random.choice([True, False])
        delivery_fee = 0
        is_free_delivery_fee =None

        if is_online_transaction:
            delivery_fee = random.randint(12000, 50000)
            is_free_delivery_fee = random.choice([True, False])

        if order_status ==2:
            payment_status = random.choices(payment_status_list, weights=payment_status_list_weights, k=1)[0]
            if payment_status ==2:
                shipping_status = random.choices(shipping_status_list, weights=shipping_status_list_weights, k=1)[0]
        

        if not is_online_transaction:
            shipping_status = None 

        payload = {
            "id": id,
            "product_id": product_id,
            "customer_id":customer_id,
            "branch_id":branch_id,
            "quantity": quantity,
            "payment_method":payment_method,
            "order_date":order_date,
            "order_status":order_status,
            "payment_status":payment_status,
            "shipping_status":shipping_status,
            "is_online_transaction":is_online_transaction,
            "delivery_fee": delivery_fee,
            "is_free_delivery_fee": is_free_delivery_fee,
            "created_at": order_date, 
            "modified_at": None
        }

        schema = {
            "schema": {
                "fields":[
                    {"type": "int32", "optional": False, "field": "id"},
                    {"type": "int32", "optional": False, "field": "product_id"},
                    {"type": "int32", "optional": False, "field": "customer_id"},
                    {"type": "int32", "optional": False, "field": "branch_id"},
                    {"type": "int32", "optional": False, "field": "quantity"},
                    {"type": "int32", "optional": True, "field": "payment_method"},
                    {"type": "int32", "optional": True, "field": "order_date"},
                    {"type": "int32", "optional": True, "field": "order_status"},
                    {"type": "int32", "optional": True, "field": "payment_status"},
                    {"type": "int32", "optional": True, "field": "shipping_status"},
                    {"type": "string", "optional": True, "field": "is_online_transaction"},
                    {"type": "int32", "optional": True, "field": "delivery_fee"},
                    {"type": "string", "optional": True, "field": "is_free_delivery_fee"},
                    {"type": "string", "optional": True, "field": "created_at"},
                    {"type": "string", "optional": True, "field": "modified_at"},
                ],
                "optional": False,
                "name": "sales_schema",
                "type": "struct"
            },
            "payload": payload
        }
        sales_data.append(schema)
    return sales_data

def delivery_report(err, msg):
    """Callback untuk menangani hasil pengiriman Kafka"""
    if err is not None:
        print(f"❌ Gagal mengirim pesan: {err}")
    else:
        print(f"✅ Data dikirim ke Kafka: {msg.topic()} [{msg.partition()}]")

while True:
    sales_data = generate_sales_data()
    for record in sales_data:
        producer.produce(TOPIC_NAME, json.dumps(record).encode("utf-8"), callback=delivery_report)
    producer.flush()

    print(f"✅ {len(sales_data)} data berhasil dikirim ke Kafka!")
    time.sleep(3600)  # Tunggu 1 menit sebelum mengirim data baru

# if __name__ == "__main__":
#     while True:
#         sales_data = generate_sales_data()
#         for sale in sales_data:
#             producer.send(TOPIC_NAME, value=sale)
#             print(f"✅ Data sent to Kafka: {json.dumps(sale, indent=2)}")

#         time.sleep(60)  # Generate data setiap 1 menit       

✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ 5 data berhasil dikirim ke Kafka!


In [ ]:
import json 
import random 
import time
import uuid 
from datetime import datetime 
from faker import Faker
from confluent_kafka import Producer
import psycopg2

fake = Faker()
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "tbl_sales_schema_new"

producer_conf = {'bootstrap.servers': KAFKA_BROKER}
producer = Producer(producer_conf)

# Konfigurasi PostgreSQL dengan port
POSTGRES_CONFIG = {
    "host": "localhost",
    "port": 5444,  # Ganti dengan port yang sesuai
    "database": "sales_project",
    "user": "postgres",
    "password": "postgres"
}

def get_list(column, tabel):
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute(f"SELECT {column} FROM {tabel} ORDER BY {column}")
        data = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
        return data
    except Exception as e:
        print(f"❌ Gagal mengambil data dari PostgreSQL: {e}")
        return []

def get_max_id():
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute("SELECT COALESCE(MAX(id), 0) FROM tbl_sales")
        max_id = cur.fetchone()[0]
        cur.close()
        conn.close()
        return max_id
    except Exception as e:
        print(f"❌ Gagal mengambil max ID dari PostgreSQL: {e}")
        return 0

product_list = get_list('id', 'tbl_product')
customer_list = get_list('id', 'tbl_customers')
branch_list = get_list('id', 'tbl_branch')
payment_method_list = get_list('id', 'tbl_payment_method')
order_status_list = get_list('id', 'tbl_order_status')
order_status_list_weights = [0.1, 0.9]
payment_status_list = get_list('id', 'tbl_payment_status')
payment_status_list_weights = [0.05, 0.9, 0.05]
shipping_status_list = get_list('id', 'tbl_shipping_status')
shipping_status_list_weights = [0.05, 0.9, 0.05]

def generate_sales_data():
    num_rows = random.randint(1, 5)
    sales_data = []
    start_id = get_max_id()

    for i in range(num_rows):
        id = start_id + i + 1
        product_id = random.choice(product_list)
        customer_id = random.choice(customer_list)
        branch_id = random.choice(branch_list)
        quantity = random.randint(1, 5)
        payment_method = random.choice(payment_method_list)
        order_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        order_status = random.choices(order_status_list, weights=order_status_list_weights, k=1)[0]
        payment_status = None
        shipping_status = None 
        is_online_transaction = random.choice([True, False])
        delivery_fee = 0
        is_free_delivery_fee = None

        if is_online_transaction:
            delivery_fee = random.randint(12000, 50000)
            is_free_delivery_fee = random.choice([True, False])

        if order_status == 2:
            payment_status = random.choices(payment_status_list, weights=payment_status_list_weights, k=1)[0]
            if payment_status == 2:
                shipping_status = random.choices(shipping_status_list, weights=shipping_status_list_weights, k=1)[0]

        if not is_online_transaction:
            shipping_status = None 

        payload = {
            "id": id,
            "product_id": product_id,
            "customer_id": customer_id,
            "branch_id": branch_id,
            "quantity": quantity,
            "payment_method": payment_method,
            "order_date": order_date,
            "order_status": order_status,
            "payment_status": payment_status,
            "shipping_status": shipping_status,
            "is_online_transaction": is_online_transaction,
            "delivery_fee": delivery_fee,
            "is_free_delivery_fee": is_free_delivery_fee,
            "created_at": order_date, 
            "modified_at": None
        }

        schema = {
            "schema": {
                "fields": [
                    {"type": "int32", "optional": False, "field": "id"},
                    {"type": "int32", "optional": False, "field": "product_id"},
                    {"type": "int32", "optional": False, "field": "customer_id"},
                    {"type": "int32", "optional": False, "field": "branch_id"},
                    {"type": "int32", "optional": False, "field": "quantity"},
                    {"type": "int32", "optional": True, "field": "payment_method"},
                    {"type": "string", "optional": True, "field": "order_date"},
                    {"type": "int32", "optional": True, "field": "order_status"},
                    {"type": "int32", "optional": True, "field": "payment_status"},
                    {"type": "int32", "optional": True, "field": "shipping_status"},
                    {"type": "boolean", "optional": True, "field": "is_online_transaction"},
                    {"type": "int32", "optional": True, "field": "delivery_fee"},
                    {"type": "boolean", "optional": True, "field": "is_free_delivery_fee"},
                    {"type": "string", "optional": True, "field": "created_at"},
                    {"type": "string", "optional": True, "field": "modified_at"},
                ],
                "optional": False,
                "name": "sales_schema",
                "type": "struct"
            },
            "payload": payload
        }
        sales_data.append(schema)
    return sales_data

def delivery_report(err, msg):
    """Callback untuk menangani hasil pengiriman Kafka"""
    if err is not None:
        print(f"❌ Gagal mengirim pesan: {err}")
    else:
        print(f"✅ Data dikirim ke Kafka: {msg.topic()} [{msg.partition()}]")

while True:
    sales_data = generate_sales_data()
    for record in sales_data:
        producer.produce(TOPIC_NAME, json.dumps(record).encode("utf-8"), callback=delivery_report)
    producer.flush()

    print(f"✅ {len(sales_data)} data berhasil dikirim ke Kafka!")
    time.sleep(3600)  # Tunggu 1 jam sebelum mengirim data baru


✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ 5 data berhasil dikirim ke Kafka!
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ 4 data berhasil dikirim ke Kafka!
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ Data dikirim ke Kafka: tbl_sales_schema_new [0]
✅ 4 data berhasil dikirim ke Kafka!


In [1]:
import json 
import random 
import json 
import time
import uuid 
import random 
from datetime import datetime 
from faker import Faker
from confluent_kafka import Producer
import psycopg2

fake = Faker()
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "tbl_sales_schema_new2"


producer_conf = {'bootstrap.servers': KAFKA_BROKER}
producer = Producer(producer_conf)
# Konfigurasi PostgreSQL

POSTGRES_CONFIG = {
    "host": "localhost",
    "database": "sales-project",
    "user": "postgres",
    "password": "postgres"
}

def get_list(column, tabel):
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute(f"select {column} from {tabel} order by {column}")
        products = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
        return products
    except Exception as e:
        print(f"❌ Gagal mengambil produk dari PostgreSQL: {e}")
        return []
    
def get_max_id():
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cur = conn.cursor()
        cur.execute("select coalesce(max(id), 0) from tbl_sales")
        max_id = cur.fetchone()[0]
        cur.close()
        conn.close()
        return max_id
    except Exception as e:
        print(f"❌ Gagal mengambil max transaction_id dari PostgreSQL: {e}")
        return 0
    

product_list = get_list('id', 'tbl_product')
customer_list = get_list('id', 'tbl_customers')
branch_list = get_list('id', 'tbl_branch')
payment_method_list = get_list('id', 'tbl_payment_method')

order_status_list = get_list('id', 'tbl_order_status')
order_status_list_weights = [0.1, 0.9]

payment_status_list = get_list('id', 'tbl_payment_status')
payment_status_list_weights = [0.05, 0.9, 0.05]

shipping_status_list = get_list('id', 'tbl_shipping_status')
shipping_status_list_weights = [0.05, 0.9, 0.05]

In [2]:
payment_method_list

[1, 2, 3, 4]

In [10]:

import pandas as pd 
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta

DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "sum_transactions"

# create connection to database 
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

query_employee = f"SELECT id, branch_id, name, salary  from {SCHEMA_NAME}.tbl_employee where active = 'true';"
df_employee =pd.read_sql(query_employee, engine)

# create data transaksi gaji for each month (Januari - Desember)
transactions = []
current_year = datetime.today().year

for _, row in df_employee.iterrows():
    for month in range(1,13):
        transactions.append({
            "type": "outcome",
            "sales_id": None,
            "branch_id": row["branch_id"],
            "employee_id": row["id"],
            "description": f"Monthly Salary {row["name"]}",
            "date": datetime(current_year, month, 1).strftime("%Y-%m-%d"),
            "amount": row["salary"]
        })

df_transactions = pd.DataFrame(transactions)

df_transactions.to_sql(TABLE_NAME,engine, schema= SCHEMA_NAME, if_exists="append", index=False)

print("Data done insert ...")



Data done insert ...


In [21]:
import pandas as pd 
import random 
from datetime import datetime, timedelta 

start_date = datetime(2025, 1, 2)
end_date = datetime(2025,12,31)
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"
TABLE_NAME  = "sum_transactions"

# create connection to database 
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

query_branch = f"select * from {SCHEMA_NAME}.tbl_branch;"
df_branch = pd.read_sql(query_branch, engine)


transactions =  []
current_year = datetime.today().year

current_date = start_date
while current_date <= end_date:
    for _, row in df_branch.iterrows():
        amount = random.randint(200000, 400000)
        transactions.append({
            "type": "outcome",
            "sales_id": None,
            "branch_id": row["id"],
            "description": f"Pengeluaran Operasional Cabang {row["name"]} sebesar Rp. {amount}",
            "date": current_date,
            "sales_id": None,
            "employee_id": None,
            "amount": amount
        })
    current_date += timedelta(days=1)

df_transactions = pd.DataFrame(transactions)


df_transactions.to_sql("sum_transactions", engine, schema=SCHEMA_NAME,if_exists="append", index = False)





200

In [23]:
df_transactions

,type,sales_id,branch_id,description,date,employee_id,amount
0,outcome,None,1,"Pengeluaran Operasional Cabang Taylor, Jimenez...",2025-01-02,None,391325
1,outcome,None,2,Pengeluaran Operasional Cabang Johnson-Brown s...,2025-01-02,None,390755
2,outcome,None,3,Pengeluaran Operasional Cabang Brown and Sons ...,2025-01-02,None,225329
3,outcome,None,4,Pengeluaran Operasional Cabang Carter-Powell s...,2025-01-02,None,201253
4,outcome,None,5,"Pengeluaran Operasional Cabang Hensley, Davenp...",2025-01-02,None,363096
...,...,...,...,...,...,...,...
18195,outcome,None,46,Pengeluaran Operasional Cabang Rivera Inc sebe...,2025-12-31,None,315445
18196,outcome,None,47,"Pengeluaran Operasional Cabang Gibson, Jones a...",2025-12-31,None,210824
18197,outcome,None,48,Pengeluaran Operasional Cabang Blanchard-Brown...,2025-12-31,None,346401
18198,outcome,None,49,"Pengeluaran Operasional Cabang Griffin, Campbe...",2025-12-31,None,292865


In [28]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta

DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"


engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

query_branch = f"select * from {SCHEMA_NAME}.tbl_branch;"
df_branch = pd.read_sql(query_branch, engine)

transactions = []
current_year = datetime.today().year

for _, row in df_branch.iterrows():
    for month in range(1,13):
        amount = random.randint(1000000, 2000000)
        transactions.append({
            "type": "outcome",
            "sales_id": None,
            "branch_id": row["id"],
            "employee_id": None, 
            "description": f"Pembayaran Listrik dan Air Cabang {row["name"]} sebesar Rp. {amount}",
            "date": datetime(current_year, month, 1).strftime("%Y-%m-%d"),  # Setiap tanggal 1 bulan tersebut
            "amount": amount
        })

df_transactions = pd.DataFrame(transactions)

df_transactions.to_sql("sum_transactions", engine, schema=SCHEMA_NAME,if_exists="append", index = False)


600

In [29]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta

DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"


engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

query_branch = f"select * from {SCHEMA_NAME}.tbl_branch;"
df_branch = pd.read_sql(query_branch, engine)

transactions = []
current_year = datetime.today().year

for _, row in df_branch.iterrows():
    for month in range(1,13):
        amount = random.randint(1000000, 2000000)
        transactions.append({
            "type": "outcome",
            "sales_id": None,
            "branch_id": row["id"],
            "employee_id": None, 
            "description": f"Pengeluaran Teknologi & IT {row["name"]} sebesar Rp. {amount}",
            "date": datetime(current_year, month, 1).strftime("%Y-%m-%d"),  # Setiap tanggal 1 bulan tersebut
            "amount": amount
        })

df_transactions = pd.DataFrame(transactions)

df_transactions.to_sql("sum_transactions", engine, schema=SCHEMA_NAME,if_exists="append", index = False)


600

In [ ]:
import pandas as pd 
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta

DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "sales-project"
SCHEMA_NAME = "public"


engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

query_branch = f"select * from {SCHEMA_NAME}.tbl_branch;"
df_branch = pd.read_sql(query_branch, engine)

transactions = []
current_year = datetime.today().year

def random_date(start_date, end_date):
    """Menghasilkan tanggal acak antara start_date dan end_date"""
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)

start = datetime(2025, 1, 1)
end = datetime(2025, 12, 31)

for _, row in df_branch.iterrows():
    for month in range(1,4):
        amount = random.randint(3000000, 7000000)
        transactions.append({
            "type": "outcome",
            "sales_id": None,
            "branch_id": row["id"],
            "employee_id": None, 
            "description": f"Pengeluaran Pemeliharaan Cabang {row["name"]} sebesar Rp. {amount}",
            "date": random_date(start, end),
            "amount": amount
        })

df_transactions = pd.DataFrame(transactions)

df_transactions.to_sql("sum_transactions", engine, schema=SCHEMA_NAME,if_exists="append", index = False)


150

,id,product_id,customer_id,branch_id,quantity,payment_method,order_date,order_status,payment_status,shipping_status,is_online_transaction,delivery_fee,is_free_delivery_fee,created_at,modified_at
0,1,7,28,38,10,3,2025-01-22 08:03:49,2,2.0,2.0,True,45099,True,2025-01-22 08:03:49,2025-02-20 08:03:49
1,2,4,76,42,9,2,2025-01-12 15:01:16,2,2.0,2.0,True,36488,False,2025-01-12 15:01:16,2025-02-07 15:01:16
2,3,8,2,25,1,4,2025-01-03 14:36:33,2,2.0,2.0,True,37593,False,2025-01-03 14:36:33,2025-01-04 14:36:33
3,4,3,60,2,9,4,2025-01-13 07:27:49,2,2.0,2.0,True,28176,True,2025-01-13 07:27:49,2025-02-04 07:27:49
4,5,3,90,5,9,2,2025-01-15 07:36:29,2,2.0,NaN,False,0,None,2025-01-15 07:36:29,2025-02-05 07:36:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,7556,10,27,31,7,4,2025-12-06 14:47:27,1,NaN,NaN,False,0,None,2025-12-06 14:47:27,2025-12-26 14:47:27
7556,7557,9,13,44,7,2,2025-12-18 12:14:08,1,NaN,NaN,True,13551,False,2025-12-18 12:14:08,2025-12-23 12:14:08
7557,7558,13,15,41,3,3,2025-12-20 15:13:55,2,2.0,2.0,True,49950,False,2025-12-20 15:13:55,2026-01-19 15:13:55
7558,7559,3,74,42,5,4,2025-12-25 11:19:10,2,2.0,NaN,False,0,None,2025-12-25 11:19:10,2025-12-27 11:19:10


In [ ]:
import pandas as pd
import psycopg2
import random
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Database Configuration
DB_CONFIG = {
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": "5432",
    "dbname": "sales-project",
    "schema": "public",
    "table": "sum_transactions"
}

def create_db_engine(config):
    return create_engine(f"postgresql+psycopg2://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['dbname']}")

def fetch_data(engine, query):
    return pd.read_sql(query, engine)

def insert_transactions(engine, transactions_df, config):
    transactions_df.to_sql(config["table"], engine, schema=config["schema"], if_exists="append", index=False)

def generate_salary_transactions(engine, config):
    query = f"""
        SELECT id, branch_id, name, salary 
        FROM {config['schema']}.tbl_employee 
        WHERE active = 'true';
    """
    df_employee = fetch_data(engine, query)
    current_year = datetime.today().year
    
    transactions = [
        {
            "type": "outcome",
            "sales_id": None,
            "branch_id": row["branch_id"],
            "employee_id": row["id"],
            "description": f"Monthly Salary {row['name']}",
            "date": datetime(current_year, month, 1).strftime("%Y-%m-%d"),
            "amount": row["salary"]
        }
        for _, row in df_employee.iterrows()
        for month in range(1, 13)
    ]
    insert_transactions(engine, pd.DataFrame(transactions), config)
    print("Salary transactions inserted.")

def generate_operational_expenses(engine, config, start_date, end_date):
    query = f"SELECT id, name FROM {config['schema']}.tbl_branch;"
    df_branch = fetch_data(engine, query)
    current_date = start_date
    transactions = []
    
    while current_date <= end_date:
        transactions.extend([
            {
                "type": "outcome",
                "sales_id": None,
                "branch_id": row["id"],
                "employee_id": None,
                "description": f"Pengeluaran Operasional Cabang {row['name']}",
                "date": current_date.strftime("%Y-%m-%d"),
                "amount": random.randint(200000, 400000)
            }
            for _, row in df_branch.iterrows()
        ])
        current_date += timedelta(days=1)
    
    insert_transactions(engine, pd.DataFrame(transactions), config)
    print("Operational expenses transactions inserted.")

def generate_utility_expenses(engine, config):
    query = f"SELECT id, name FROM {config['schema']}.tbl_branch;"
    df_branch = fetch_data(engine, query)
    current_year = datetime.today().year
    
    transactions = [
        {
            "type": "outcome",
            "sales_id": None,
            "branch_id": row["id"],
            "employee_id": None,
            "description": f"Pembayaran Listrik dan Air Cabang {row['name']}",
            "date": datetime(current_year, month, 1).strftime("%Y-%m-%d"),
            "amount": random.randint(1000000, 2000000)
        }
        for _, row in df_branch.iterrows()
        for month in range(1, 13)
    ]
    insert_transactions(engine, pd.DataFrame(transactions), config)
    print("Utility expenses transactions inserted.")

if __name__ == "__main__":
    engine = create_db_engine(DB_CONFIG)
    generate_salary_transactions(engine, DB_CONFIG)
    generate_operational_expenses(engine, DB_CONFIG, datetime(2025, 1, 2), datetime(2025, 12, 31))
    generate_utility_expenses(engine, DB_CONFIG)
